In [27]:
import torch

In [29]:
# Russian to English translation
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en', checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
                       tokenizer='moses', bpe='fastbpe')

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


In [30]:
ru2en = ru2en.cuda()

In [31]:
ru2en.eval()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(31232, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear(in_feat

In [32]:
with open('eval-ru-100.txt') as f:
    lines = f.readlines()

In [34]:
from tqdm.notebook import tqdm

In [36]:
with open('answer.txt', 'a+') as the_file:
    russian = []
    for line in tqdm(lines):
        translation = ru2en.translate(line, beam=5, verbose=True)
        russian.append(translation)
        the_file.write(translation+"\n")

  0%|          | 0/100 [00:00<?, ?it/s]

# Try smart inference

In [8]:
block_i = -1
for line in lines:
    first_word = line.split(" ")[0]
    if first_word[:-1].isdigit():
        block_i += 1
        nums_blocks.append(first_word)
        sent_blocks.append([line[len(first_word) + 1:]])
    else:
        sent_blocks[block_i].append(line)

In [9]:
text_blocks = [''.join(block) for block in sent_blocks]

In [10]:
translated = []

In [11]:
for block in tqdm(text_blocks):
    translation = ru2en.translate(block)
    translated.append(translation)

  0%|          | 0/18 [00:00<?, ?it/s]

In [75]:
numbered_translated = []

In [76]:
for block_num, translation in zip(nums_blocks, translated):
    numbered_translation = translation.copy()
    numbered_translation[0] = f"{block_num} {numbered_translation[0]}"
    numbered_translation[1:] = [f"{trans}\n" for trans in numbered_translation[1:]]
    numbered_translated.append(numbered_translation)

In [77]:
numbered_translated = [''.join(trans) for trans in numbered_translated]